In [16]:
import torch
from markov_bridges.models.generative_models.cmb import CMB
from markov_bridges.configs.config_classes.generative_models.cmb_config import CMBConfig
from markov_bridges.configs.config_classes.data.basics_configs import IndependentMixConfig
from markov_bridges.configs.config_classes.trainers.trainer_config import CMBTrainerConfig
from markov_bridges.configs.config_classes.pipelines.pipeline_configs import CMBPipelineConfig 
from markov_bridges.configs.config_classes.networks.mixed_networks_config import MixedDeepMLPConfig

# Configuration

In [7]:
model_config = CMBConfig(continuous_loss_type="drift")
model_config.data = IndependentMixConfig(has_context_discrete=True)
model_config.mixed_network = MixedDeepMLPConfig(time_embed_dim=19,
                                                num_layers=1,
                                                discrete_embed_dim=20)
model_config.pipeline = CMBPipelineConfig(number_of_steps=10)
model_config.trainer = CMBTrainerConfig(number_of_epochs=20)

# Create Model

In [8]:
cmb = CMB(model_config,device=torch.device("cpu"))

c:\Users\cesar\anaconda4\envs\rate_matching\lib\site-packages\torch\nn\init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [20]:
float(False)

0.0

## Handle Data

In [9]:
databatch = cmb.dataloader.get_databatch()

In [10]:
databatch_dict = databatch._asdict()

In [11]:
print(databatch_dict.keys())

dict_keys(['source_continuous', 'target_continuous', 'context_discrete', 'time'])


# Model Bridge


In [12]:
discrete_sample,continuous_sample = cmb.forward_map.sample_bridge(databatch)